#Imports

In [ ]:
%tensorflow_version 1.x
%load_ext tensorboard

import tensorflow as tf
from tensorflow import keras

from tensorflow.python.keras.models import *

from tensorflow.python.keras.layers import *
from tensorflow.python.keras.optimizers import *

# from keras.models import *
# from keras.layers import *
# from keras.optimizers import *

from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorboard import notebook

#from tensorflow.contrib.learn.python.learn.datasets.mnist import extract_images, extract_labels

print('done imports')

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
done imports
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
done imports


In [ ]:
# https://www.depends-on-the-definition.com/unet-keras-segmenting-images/
# https://github.com/zhixuhao/unet/blob/master/model.py
# https://ai.googleblog.com/2020/01/using-machine-learning-to-nowcast.html
# Input -> 1Basic -> 7 DownSample -> 1Basic -> 7 UpSample

# LOAD GOOGLE DRIVE IMAGES and populate inputs

In [ ]:
from google.colab import drive
from os import listdir
from os.path import isfile, join
import sys
import numpy as np
from PIL import Image 
from matplotlib.pyplot import imshow

np.set_printoptions(threshold=sys.maxsize)

# %debug
drive.mount('/content/drive/', force_remount=True)
path = '/content/drive/My Drive/Skynet/WVY'
path_out = '/content/drive/My Drive/Skynet/WVY_SCRUB'
# listdir(path)



# https://www.youtube.com/watch?v=2pQOXjpO_u0 creating our own dataset
#current image mode for gif input is P, meaning 1 bytes per pixel

Mounted at /content/drive/


### Run following 2 sections to load images from google drive

In [ ]:
files_in_dir = [f for f in listdir(path) if isfile(join(path, f)) if f.split('_')[4] == 'RAIN.gif']


print(len(files_in_dir))
# for f in files_in_dir:
#     print(f)

4852


In [ ]:
import matplotlib.pyplot as plt
import cv2
import math

numImages = 120*40
background_image = plt.imread('background.gif')

count = 0
rain_images = []
files_in_dir = files_in_dir[:numImages]  # Reduce number of images for now to 60


for f in files_in_dir:  # Clean and crop each image
    s = join(path, files_in_dir[count])
    count += 1

    rain_image = plt.imread(s)
    height, width, depth = rain_image.shape

    imgdiff = background_image - rain_image # Perform background suppression
    imggood = cv2.cvtColor(imgdiff, cv2.COLOR_BGR2RGB) # Switch back to normal color encoding

    cropped = imggood[0:480, 0:480]  # Crop
    height, width, depth = cropped.shape
    resized = cv2.resize(cropped, (int(height / 7.5), int(width / 7.5)), interpolation = cv2.INTER_AREA)  # Resize to 64 by 64
    converted = cv2.cvtColor(resized, cv2.COLOR_RGB2GRAY)
    converted = converted[..., np.newaxis]
    rain_images.append(np.asarray(converted))  # Add to new rain images list

# imshow(rain_images[0])  # Test output
# half_images = numImages // 2
half_images = math.floor(numImages * 0.8)
train_list = rain_images[:half_images]  # Split into 30 train and 30 test
test_list = rain_images[half_images:]

print('done loading images and converting to greyscale')

train_images = []
train_images_extra = []
train_labels = []
for i in range (0, len(train_list)):  # Split train into 20 images and 10 labels
  if i % 3 == 2:
    train_labels.append(np.ndarray.flatten(train_list[i]))
  elif i % 3 == 1:
    train_images_extra.append(train_list[i])
  else:
    train_images.append(train_list[i])


test_images = []
test_images_extra = []
test_labels = []
for i in range (0, len(test_list)):  # Split test into 20 images and 10 labels
  if i % 3 == 2:
    test_labels.append(np.ndarray.flatten(test_list[i]))
  elif i % 3 == 1:
    test_images_extra.append(test_list[i])
  else:
    test_images.append(test_list[i])

print('done separating training/testing images and labels')

good_train_images = []
for i in range(0, len(train_labels)):
  image = np.concatenate((train_images[i], train_images_extra[i]), 0)
  image = np.expand_dims(image, axis=0)
  good_train_images.append(image)

good_test_images = []
for i in range(0, len(test_labels)):
  image = np.concatenate((test_images[i], test_images_extra[i]), 0)
  image = np.expand_dims(image, axis=0)
  good_test_images.append(image)



good_train_images = np.concatenate((good_train_images), 0)
print(good_train_images.shape)

train_labels = np.asarray(train_labels)
print(train_labels.shape)

good_test_images = np.concatenate((good_test_images), 0)
print(good_test_images.shape)

test_labels = np.asarray(test_labels)
print(test_labels.shape)
#train_labels = np.concatenate((train_labels), 0)


#print(train_labels.shape)

print('done')

done loading images and converting to greyscale
done separating training/testing images and labels
(1280, 128, 64, 1)
(1280, 4096)
(320, 128, 64, 1)
(320, 4096)
done


#### Run this to save the current arrays as files - should make it faster to load the sets in the future

In [ ]:
np.save('/content/drive/My Drive/Skynet/training_images.npy', good_train_images)
np.save('/content/drive/My Drive/Skynet/training_labels.npy', train_labels)
np.save('/content/drive/My Drive/Skynet/testing_images.npy', good_test_images)
np.save('/content/drive/My Drive/Skynet/testing_labels.npy', test_labels)
print('done')

done


## Only need to run this to load the training and test data

In [ ]:
good_train_images = np.load('/content/drive/My Drive/Skynet/training_images.npy')
train_labels = np.load('/content/drive/My Drive/Skynet/training_labels.npy')
good_test_images = np.load('/content/drive/My Drive/Skynet/testing_images.npy')
test_labels = np.load('/content/drive/My Drive/Skynet/testing_labels.npy')

print(good_train_images.shape)
print(train_labels.shape)
print(good_test_images.shape)
print(test_labels.shape)

(1280, 128, 64, 1)
(1280, 4096)
(320, 128, 64, 1)
(320, 4096)


In [ ]:
!pip install varname

from varname import nameof


def print_shape(variable_in, name):
    print('{} shape: {}'.format(name, variable_in.shape))

# Model

### Setting up model

In [ ]:

    # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
def loss(y_true,y_pred):
    # return K.mean(K.square(y_pred - y_true), axis=-1)
    return (y_true - y_pred)**2


In [ ]:
# from tensorflow.python.keras import Model
# code from here https://github.com/zhixuhao/unet
#concatenate basically adds one image to the end of the other for each image the 2 layers (keeping each of the 2 added ones seperate)
# https://keras.io/api/layers/merging_layers/concatenate/


input1 = keras.Input((128, 64, 1))
print_shape(input1, nameof(input1))

# input2 = Input((64, 64, 1))
# print_shape(input2, 'input 2')

# inputs_avg = Average()([input1, input2])
# inputs_avg = Concatenate()([input1, input2])

#basic 1 start

conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(input1)
print_shape(conv1, nameof(conv1))

batchNorm1 = BatchNormalization(axis=3)(conv1)
print_shape(batchNorm1, 'batch norm 1')

leakyRelu1 = LeakyReLU(alpha=0.3)(batchNorm1)
print_shape(leakyRelu1, nameof(leakyRelu1))

conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(leakyRelu1)
print_shape(conv1, nameof(conv1))

# basic 1 end

#down-sample 1 start

batchNorm2 = BatchNormalization(axis=3)(conv1)
print_shape(batchNorm2, 'batch norm 2')

leakyRelu2 = LeakyReLU(alpha=0.3)(batchNorm2)
print_shape(leakyRelu2, nameof(leakyRelu2))

pool1 = MaxPooling2D(pool_size=(2, 2))(leakyRelu2)
print_shape(pool1, nameof(pool1))

batchNorm3 = BatchNormalization(axis=3)(pool1)
print_shape(batchNorm3, 'batch norm 3')

leakyRelu3 = LeakyReLU(alpha=0.3)(batchNorm3)
print_shape(leakyRelu3, nameof(leakyRelu3))

conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(leakyRelu3)
print_shape(conv2, nameof(conv2))

#down-sample 1 end
#down-sample 2 start

# conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
# print_shape(conv2, nameof(conv2))

batchNorm4 = BatchNormalization(axis=3)(conv2)
print_shape(batchNorm4, 'batch norm 4')

leakyRelu4 = LeakyReLU(alpha=0.3)(batchNorm4)
print_shape(leakyRelu4, nameof(leakyRelu4))

pool2 = MaxPooling2D(pool_size=(2, 2))(leakyRelu4)
print_shape(pool2, nameof(pool2))

batchNorm5 = BatchNormalization(axis=3)(pool2)
print_shape(batchNorm5, 'batch norm 5')

leakyRelu5 = LeakyReLU(alpha=0.3)(batchNorm5)
print_shape(leakyRelu5, nameof(leakyRelu5))

conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(leakyRelu5)
print_shape(conv3, nameof(conv3))

# down-sample 2 end
# down-sample 3 start

# conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
# print_shape(conv3, nameof(conv3))

batchNorm6 = BatchNormalization(axis=3)(conv3)
print_shape(batchNorm1, 'batch norm 1')

leakyRelu6 = LeakyReLU(alpha=0.3)(batchNorm6)
print_shape(leakyRelu6, nameof(leakyRelu6))

pool3 = MaxPooling2D(pool_size=(2, 2))(leakyRelu6)
print_shape(pool3, nameof(pool3))

batchNorm7 = BatchNormalization(axis=3)(pool3)
print_shape(batchNorm7, 'batch norm 7')

leakyRelu7 = LeakyReLU(alpha=0.3)(batchNorm7)
print_shape(leakyRelu7, nameof(leakyRelu7))

conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(leakyRelu7)
print_shape(conv4, nameof(conv4))

# conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
# print_shape(conv4, nameof(conv4))

# down-sample 3 end

# down-sample 4 start

# drop4 = Dropout(0.5)(conv4)
# print_shape(drop4, nameof(drop4))

batchNorm8 = BatchNormalization(axis=3)(conv4)
print_shape(batchNorm1, 'batch norm 8')

leakyRelu8 = LeakyReLU(alpha=0.3)(batchNorm8)
print_shape(leakyRelu8, nameof(leakyRelu8))

pool4 = MaxPooling2D(pool_size=(2, 2))(leakyRelu8)
print_shape(pool4, nameof(pool4))

batchNorm9 = BatchNormalization(axis=3)(pool4)
print_shape(batchNorm9, 'batch norm 9')

leakyRelu9 = LeakyReLU(alpha=0.3)(batchNorm9)
print_shape(leakyRelu9, nameof(leakyRelu9))

conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(leakyRelu9)
print_shape(conv5, nameof(conv5))

# down-sample 4 end

# conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
# print_shape(conv5, nameof(conv5))

# drop5 = Dropout(0.5)(conv5)
# print_shape(drop5, nameof(drop5))

# TODO: PUT BASIC LAYER HERE

conv_basic1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
print_shape(conv_basic1, nameof(conv_basic1))

batchNorm_basic = BatchNormalization(axis=3)(conv_basic1)
print_shape(batchNorm_basic, 'batch norm _basic')

leakyRelu_basic = LeakyReLU(alpha=0.3)(batchNorm_basic)
print_shape(leakyRelu_basic, nameof(leakyRelu_basic))

conv_basic2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(leakyRelu_basic)
print_shape(conv_basic2, nameof(conv_basic2))

# up-sample 1 start

upa6 = UpSampling2D(size = (2,2))(conv_basic2)
print_shape(upa6, nameof(upa6))

batchNorm10 = BatchNormalization(axis=3)(upa6)
print_shape(batchNorm10, 'batch norm 10')

leakyRelu10 = LeakyReLU(alpha=0.3)(batchNorm10)
print_shape(leakyRelu10, nameof(leakyRelu10))

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(leakyRelu10)
print_shape(up6, nameof(up6))

batchNorm11 = BatchNormalization(axis=3)(up6)
print_shape(batchNorm11, 'batch norm 11')

leakyRelu11 = LeakyReLU(alpha=0.3)(batchNorm11)
print_shape(leakyRelu11, nameof(leakyRelu11))

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(leakyRelu11)
print_shape(up6, nameof(up6))

# up-sample 1 end

# up-sample 2 start

merge6 = concatenate([conv4,up6], axis = 3)
# merge6 = concatenate([drop4,up6], axis = 3)
print_shape(merge6, nameof(merge6))

# conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
# print_shape(conv6, nameof(conv6))

upa7 = UpSampling2D(size = (2,2))(merge6)
print_shape(upa7, nameof(upa7))

batchNorm12 = BatchNormalization(axis=3)(upa7)
print_shape(batchNorm12, 'batch norm 12')

leakyRelu12 = LeakyReLU(alpha=0.3)(batchNorm12)
print_shape(leakyRelu12, nameof(leakyRelu12))


up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(leakyRelu12)
print_shape(up7, nameof(up7))

batchNorm13 = BatchNormalization(axis=3)(up7)
print_shape(batchNorm13, 'batch norm 13')

leakyRelu13 = LeakyReLU(alpha=0.3)(batchNorm13)
print_shape(leakyRelu13, nameof(leakyRelu13))

up7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(leakyRelu13)
print_shape(up7, nameof(up7))

# up-sample 2 end

merge7 = concatenate([conv3,up7], axis = 3)
print_shape(merge7, nameof(merge7))

# up-sample 3 start



# conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
# print_shape(conv7, nameof(conv7))

upa8 = UpSampling2D(size = (2,2))(merge7)

batchNorm14 = BatchNormalization(axis=3)(upa8)
print_shape(batchNorm14, 'batch norm 14')

leakyRelu14 = LeakyReLU(alpha=0.3)(batchNorm14)
print_shape(leakyRelu14, nameof(leakyRelu14))

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(leakyRelu14)
print_shape(up8, nameof(up8))

batchNorm15 = BatchNormalization(axis=3)(up8)
print_shape(batchNorm15, 'batch norm 15')

leakyRelu15 = LeakyReLU(alpha=0.3)(batchNorm15)
print_shape(leakyRelu15, nameof(leakyRelu15))

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(leakyRelu15)
print_shape(up8, nameof(up8))

# up-sample 3 end
# up-sample 4 start

merge8 = concatenate([conv2,up8], axis = 3)
print_shape(merge8, nameof(merge8))

# conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
# print_shape(conv8, nameof(conv8))

# conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
# print_shape(conv8, nameof(conv8))

upa9 = UpSampling2D(size = (2,2))(merge8)

batchNorm13 = BatchNormalization(axis=3)(upa9)
print_shape(batchNorm13, 'batch norm 13')

leakyRelu13 = LeakyReLU(alpha=0.3)(batchNorm13)
print_shape(leakyRelu13, nameof(leakyRelu13))

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(leakyRelu13)
print_shape(up9, nameof(up9))

batchNorm13 = BatchNormalization(axis=3)(up9)
print_shape(batchNorm13, 'batch norm 13')

leakyRelu13 = LeakyReLU(alpha=0.3)(batchNorm13)
print_shape(leakyRelu13, nameof(leakyRelu13))

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(leakyRelu13)
print_shape(up9, nameof(up9))

merge9 = concatenate([conv1,up9], axis = 3)
print_shape(merge9, nameof(merge9))

# BASIC FROM HERE ON
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
print_shape(conv9, nameof(conv9))

batchNorm14 = BatchNormalization(axis=3)(conv9)
print_shape(batchNorm13, 'batch norm 13')

leakyRelu14 = LeakyReLU(alpha=0.3)(batchNorm14)
print_shape(leakyRelu14, nameof(leakyRelu14))

conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(leakyRelu14)
print_shape(conv9, nameof(conv9))

conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
print_shape(conv9, nameof(conv9))

conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
print_shape(conv10, nameof(conv10))
print('conv 10 shape: ', conv10.shape)


flated = Flatten()(conv10)
print('flat shape: ', flated.shape)

dense_penultimate = Dense(4096, activation='relu')(flated)
print_shape(dense_penultimate, nameof(dense_penultimate))

# dense_end = Dense(8)(dense_penultimate) # 8 classes at end, one for each rain colour amount
# print_shape(dense_end, nameof(dense_end))


model_online = Model(inputs = input1, outputs = dense_penultimate)

# model_online.compile(optimizer ='adamax', loss = loss, metrics = ['accuracy', 'poisson'])
model_online.compile(optimizer ='adamax', loss = 'binary-crossentropy', metrics = ['accuracy', 'poisson'])

model_online.summary()

input1 shape: (?, 128, 64, 1)
conv1 shape: (?, 128, 64, 64)
batch norm 1 shape: (?, 128, 64, 64)
leakyRelu1 shape: (?, 128, 64, 64)
conv1 shape: (?, 128, 64, 64)
batch norm 2 shape: (?, 128, 64, 64)
leakyRelu2 shape: (?, 128, 64, 64)
pool1 shape: (?, 64, 32, 64)
batch norm 3 shape: (?, 64, 32, 64)
leakyRelu3 shape: (?, 64, 32, 64)
conv2 shape: (?, 64, 32, 128)
batch norm 4 shape: (?, 64, 32, 128)
leakyRelu4 shape: (?, 64, 32, 128)
pool2 shape: (?, 32, 16, 128)
batch norm 5 shape: (?, 32, 16, 128)
leakyRelu5 shape: (?, 32, 16, 128)
conv3 shape: (?, 32, 16, 256)
batch norm 1 shape: (?, 128, 64, 64)
leakyRelu6 shape: (?, 32, 16, 256)
pool3 shape: (?, 16, 8, 256)
batch norm 7 shape: (?, 16, 8, 256)
leakyRelu7 shape: (?, 16, 8, 256)
conv4 shape: (?, 16, 8, 512)
batch norm 8 shape: (?, 128, 64, 64)
leakyRelu8 shape: (?, 16, 8, 512)
pool4 shape: (?, 8, 4, 512)
batch norm 9 shape: (?, 8, 4, 512)
leakyRelu9 shape: (?, 8, 4, 512)
conv5 shape: (?, 8, 4, 1024)
conv_basic1 shape: (?, 8, 4, 64)
batc

### Training model

In [ ]:
!rm -rf ./logs/ 

logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)



# train_images = np.stack(train_images, axis=0)
# train_labels = np.stack(train_labels, axis=0)

model_online.fit(
    good_train_images,
    train_labels, 
    batch_size=16,
    epochs=3,
    verbose=1,
    validation_split = 0.1,
    callbacks=[tensorboard_callback])

%tensorboard --logdir logs

Train on 1152 samples, validate on 128 samples
Epoch 1/3
1152/1152 [==============================] - 995s 864ms/sample - loss: 14077.2784 - acc: 8.6806e-04 - poisson: 655.7801 - val_loss: 11684.6587 - val_acc: 0.0000e+00 - val_poisson: 545.7925
Epoch 2/3
1152/1152 [==============================] - 981s 852ms/sample - loss: 10278.4699 - acc: 0.0000e+00 - poisson: 528.6441 - val_loss: 11102.7646 - val_acc: 0.0000e+00 - val_poisson: 520.9187
Epoch 3/3
 992/1152 [========================>.....] - ETA: 2:13 - loss: 10129.4240 - acc: 0.0000e+00 - poisson: 517.6982